# 과의존 데이터로 분류 학습 정확도 확인

In [1]:
import pandas as pd
import pymysql
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

## 테스트는 성인(DM1 == 3) 인 데이터로 진행(양이 가장 많기 때문에)

#### DB 접속정보 정의

In [2]:
# db 접속정보 설정
DB_HOST=''
DB_PORT = ''
DB_USER = ''
DB_PASS = ''
DB_NAME=''

#### DB 에서 데이터 조회하여 DataFrame 으로 변환

In [4]:
# db 접속
conn = pymysql.connect(host = DB_HOST, port = DB_PORT, user = DB_USER, passwd = DB_PASS, db = DB_NAME, cursorclass=pymysql.cursors.DictCursor)
cur = conn.cursor()

sql = """
SELECT b.* 
  FROM respondent a
 INNER JOIN response_fit b on a.ID = b.ID 
 WHERE a.DM1 = 3
"""
cur.execute(sql)

result = cur.fetchall()
df = pd.DataFrame(result)

# db 연결 종료
conn.commit()    
conn.close()

#### feature 와 label 분리

In [10]:
df_y = df['KK1']
df_X = df.drop(['ID', 'KK1'], axis=1)

#### train set 와 test set 분할

In [14]:
X_train, X_test, y_train, y_test = train_test_split(df_X.values, df_y.values)

#### 정확도 계산결과 확인

In [15]:
lr_c = LogisticRegression(max_iter=4000)
lr_c.fit(X_train, y_train)
pred = lr_c.predict(X_test)
accuracy_score(y_test, pred)

0.9987080103359173

### 다른 그룹 정확도를 확인하기 위해 함수를 만들어서 처리

In [16]:
def accuracy(dm1):
    
    # DB 에서 데이터 읽어오기
    # db 접속
    conn = pymysql.connect(host = DB_HOST, port = DB_PORT, user = DB_USER, passwd = DB_PASS, db = DB_NAME, cursorclass=pymysql.cursors.DictCursor)
    cur = conn.cursor()

    sql = """
    SELECT b.* 
      FROM respondent a
     INNER JOIN response_fit b on a.ID = b.ID 
     WHERE a.DM1 = {}
    """.format(dm1)
    cur.execute(sql)

    result = cur.fetchall()
    df = pd.DataFrame(result)

    # db 연결 종료
    conn.commit()    
    conn.close()
    
    # feature 와 label 분리
    df_y = df['KK1']
    df_X = df.drop(['ID', 'KK1'], axis=1)
    
    # 1그룹의 경우 문항이 9개밖에 없으므로 Q3_10 컬럼 제거해주어야 함
    if dm1 == 1:
        df_X = df_X.drop(['Q3_10'], axis=1)
    
    # train set, test set 분리
    X_train, X_test, y_train, y_test = train_test_split(df_X.values, df_y.values)
    
    # 정확도 계산결과 확인
    lr_c = LogisticRegression(max_iter=4000)
    lr_c.fit(X_train, y_train)
    pred = lr_c.predict(X_test)
    score = accuracy_score(y_test, pred)
    
    print('{} 그룹 정확도 : {}'.format(dm1, score))
    

In [17]:
accuracy(1)
accuracy(2)
accuracy(3)
accuracy(4)

1 그룹 정확도 : 1.0
2 그룹 정확도 : 0.9885433715220949
3 그룹 정확도 : 0.9987080103359173
4 그룹 정확도 : 0.998003992015968
